In [ ]:
import os
%load_ext rpy2.ipython

In [ ]:
%%R
workDir = '/home/chantal/Chazy/Bulk/ITS/data/'
figDir = '/home/chantal/Chazy/Bulk/ITS/data/figs/'

physeqDir = '/home/chantal/Chazy/data/phyloseq/ITS/'
physeqBulk = 'bulk-core'


nprocs = 20

In [ ]:
%%R
library(ggplot2)
library(phyloseq)
library(tidyr)
library(plyr);library(dplyr)

library(scales)
library(biom)
library(metagenomeSeq)
library(doParallel)


In [ ]:
%%R
F = file.path(physeqDir, physeqBulk)
physeq.Bulk = readRDS(F)
physeq.Bulk.m = physeq.Bulk %>% sample_data
#physeq.Bulk.Sparsity = filter_taxa(physeq.Bulk, function(x) sum(x > 0) > (0.25 * length(x)), TRUE)

physeq.Bulk.m = physeq.Bulk %>% sample_data

physeq.Bulk.core = prune_samples(physeq.Bulk.m$Land_Management != "PAS", physeq.Bulk)

physeq.Bulk.core = physeq.Bulk.core %>% filter_taxa(function(x) sum(x) > 0, TRUE)

physeq.Bulk.m = physeq.Bulk.core %>% sample_data
    

#print(physeq.Bulk.Sparsity)
print(physeq.Bulk.core)


In [ ]:
%%R
physeq.Bulk.m = physeq.Bulk.core %>% sample_data

In [ ]:
%%R
physeq.Bulk.m$Sample_Date = as.POSIXct(strptime(physeq.Bulk.m$Sample_Date, format = "%m/%d/%y"))
str(physeq.Bulk.m$Sample_Date)

In [ ]:
%%R
physeq.Bulk.m$Day = round(difftime(physeq.Bulk.m$Sample_Date, min(physeq.Bulk.m$Sample_Date), 
         units = "days"))
tail(physeq.Bulk.m)

In [ ]:
%%R
physeq.Bulk.m$Day_rel = as.numeric(physeq.Bulk.m$Day)


In [ ]:
%%R
physeq.final = merge_phyloseq(physeq.Bulk.core, physeq.Bulk.m)
physeq.final

In [ ]:
%%R
make_metagenomeSeq = function(physeq) {
    require("metagenomeSeq")
    require("phyloseq")
    # Enforce orientation
    if (!taxa_are_rows(physeq)) {
        physeq <- t(physeq)
    }
    OTU = as(otu_table(physeq), "matrix")
    #OTUTill = subset(OTU, rownames(OTU) %in% df.Till.r$OTU)

    # Convert sample_data to AnnotatedDataFrame
    ADF = AnnotatedDataFrame(data.frame(sample_data(physeq)))
    # define dummy 'feature' data for OTUs, using their name Helps with
    # extraction and relating to taxonomy later on.
    TDF = AnnotatedDataFrame(data.frame(tax_table(physeq)))
    #TDFTill = subset(TDF, rownames(TDF) %in% df.Till.r$OTU)
    TDF$Rank9 = rownames(TDF)


    # Create the metagenomeSeq object
    MGS = newMRexperiment(counts = OTU, phenoData = ADF, featureData = TDF)
    # Trigger metagenomeSeq to calculate its Cumulative Sum scaling factor.
    MGS = cumNorm(MGS)
    return(MGS)
}

MR = make_metagenomeSeq(physeq.final)

In [ ]:
%%R
str(MR)

In [ ]:
%%R
TimeSeries = function(MR, feature) {  
    
    res = fitTimeSeries(obj = MR, lvl = 'Rank9', feature = feature, class = "Till",
                    id = "Full.sample", time = "Day_rel", log = TRUE)

    return(res)
}




In [ ]:
%%R
OTU = as(otu_table(physeq.final), "matrix")

feature = rownames(OTU)
str(feature)

In [ ]:
%%R
mdf = psmelt(physeq.final)

In [ ]:
%%R
test2 = filter(mdf, OTU == 'OTU.12348') 
summary(test2$Abundance)

In [ ]:
%%R
test2 %>% group_by(Abundance) %>% summarise(sum = n())

In [ ]:
%%R
range(test$count)

In [ ]:
%%R
feature[1766]

In [ ]:
%%R
feature = feature[-627]
feature = feature[-13]
feature = feature[-1147]
feature = feature[-1415]
feature = feature[-1767]

In [ ]:
%%R
#OTUs not found across all timepoints?
registerDoParallel(10)

TS = llply(feature, 
          TimeSeries, 
          .parallel = TRUE,
          MR = MR 
         )

In [ ]:
%%R
str(TS)

In [ ]:
%%R
names(TS) = feature

In [ ]:
%%R
timeSeriesFits = sapply(TS,function(i){i[[1]]})[-grep("No",TS)]

In [ ]:
%%R
str(timeSeriesFits)

In [ ]:
%%R
for(i in 1:length(timeSeriesFits.Filt)){
    rownames(timeSeriesFits.Filt[[i]]) =
    paste(
    paste(names(timeSeriesFits.Filt)[i]," interval",sep=""),
    1:nrow(timeSeriesFits.Filt[[i]]),sep=":"
)
}

In [ ]:
%%R
timeSeriesFits = as.data.frame(do.call(rbind, timeSeriesFits.Filt))


#do.call(rbind,timeSeriesFits)

In [ ]:
%%R
timeSeriesFits = as.data.frame(do.call(rbind, timeSeriesFits.Filt))


#do.call(rbind,timeSeriesFits)

In [ ]:
%%R
pvalues = timeSeriesFits[,"p.value"]
adjPvalues = p.adjust(pvalues,"bonferroni")
timeSeriesFits = cbind(timeSeriesFits,adjPvalues)
head(timeSeriesFits)

In [ ]:
%%R
write.csv(timeSeriesFits, 'data/timeSeries_tillage_all_ITS.csv')

In [ ]:
%%R
TSTill = read.csv('data/timeSeries_tillage_all_ITS.csv')

In [ ]:
%%R
filter(TSTill, X == 'OTU.10 interval:1')

In [ ]:
%%R
TS.sig = filter(TSTill, adjPvalues <= 0.01)
length(TS.sig$X)

In [ ]:
%%R
TS.sig$OTU_interval = TS.sig$X
TS.sig = separate(TS.sig, OTU_interval, c('OTU', 'interval'), sep = " ") %>% separate(interval, c('interval', 'num'))
head(TS.sig)

In [ ]:
%%R
print(length(unique(TS.sig$OTU))) 
Mult_resp = TS.sig %>% group_by(OTU) %>% summarise(respTime = n()) %>% filter(respTime > 1)

In [ ]:
%%R
Make_DF = function(TS) {  
    
    d = data.frame(TS$data[, c("abundance","class", "time", "id")])
    d$Sample = rownames(TS$data)


    return(d)
}

In [ ]:
%%R
l = list()
for (i in 1:length(feature)) {
    classname = as.vector(feature[i])
    l[[classname]] = Make_DF(TS[[i]])
    
}

In [ ]:
%%R
DF = do.call(rbind, l)

In [ ]:
%%R
write.csv(DF, 'data/TimeSeries_Abundance_Till_ITS.csv')

In [ ]:
%%R
DF = read.csv('data/TimeSeries_Abundance_Till_ITS.csv')
head(DF)

In [ ]:
%%R
head(DF)

In [ ]:
%%R
DF2 = separate(DF, Class_Sample, c('OTU', "num", "MY", "Treat", 'Rep'), sep = '\\.' )
DF2$C = paste(DF2$OTU, DF2$num, sep = '.')
DF2$Full.sample = paste(DF2$MY, DF2$Treat, DF2$Rep, sep = '.')
DF2 = filter(DF2, C %in% TS.sig$OTU)
head(DF2)

In [ ]:
%%R
DF2$Treat_real = substr(DF2$Treat, 1, 3)
head(DF2)

In [ ]:
%%R -w 1000 -h 400
TS.sig$ISIE = paste(TS.sig$Interval.start, TS.sig$Interval.end, sep = '_')
MResp = filter(TS.sig, OTU %in% Mult_resp$OTU) %>% group_by(OTU)

MResp$ISIE = factor(MResp$ISIE, levels = MResp$ISIE[order(MResp$OTU)])


ggplot(MResp, aes(x = ISIE, group = OTU)) + geom_bar(aes(fill = OTU))  + 
        theme(text = element_text(size=18),
        axis.title.y = element_text(vjust=1),
        axis.text.x = element_text(angle=90, vjust=0.5, hjust=1)) + xlab(NULL) + 
        ylab('Number of Time Periods')

In [ ]:
%%R
head(DF2)
DF.sum = DF2 %>% group_by(C, time, class, Treat_real) %>% summarise(meanab = mean(abundance), sdab = sd(abundance))

In [ ]:
%%R
head(DF.sum)

In [ ]:
%%R
Nresp = DF.sum %>% group_by(C, class) %>% summarise(ab = sum(meanab)) 

In [ ]:
%%R
Seas = Mult_TS %>% filter(type == 'Seasonal') %>% ungroup %>% arrange(Area) 

In [ ]:
%%R -h 300 -w 800
example = 'OTU.15234'

TS.sig$C = TS.sig$OTU

DF.filt = filter(DF.sum, C %in% example)
TS.filt = filter(TS.sig, C %in% example )

limits = aes(ymax = meanab + sdab, ymin=meanab -sdab, color = Treat_real)
ggplot(DF.filt, aes(x = time, y = meanab)) + 
geom_point(aes(color = Treat_real), size = 3) + 
geom_errorbar(limits, width=0.25)+
geom_segment(data = TS.filt, aes(x = Interval.start, xend = Interval.end, y = 10, yend = 10)) + 
labs(title = example) + theme(text = element_text(size=16)) + ylab('Abundance') + xlab('Day') +
scale_color_discrete(name = "Land Management") + facet_wrap(~C, ncol = 7)

In [ ]:
%%R
filter(TS.sig, OTU == 'OTU.51')

In [ ]:
%%R
head(DF.sum)

In [ ]:
%%R
head(TS.sig)
TS.sig$Start[TS.sig$Interval.start %in% c(0:75)] = 'July2014'
#TS.sig$Start[TS.sig$Interval.start > 0 & TS.sig$Interval.start <76] = 'JunetoSept_2014'
TS.sig$Start[TS.sig$Interval.start %in% c(76:110)] = 'September2014'
#TS.sig$Start[TS.sig$Interval.start > 76 & TS.sig$Interval.start < 111 ] = 'SepttoOct_2014'
TS.sig$Start[TS.sig$Interval.start %in% c(111:138)] = 'October2014'
#TS.sig$Start[TS.sig$Interval.start > 111 & TS.sig$Interval.start < 139 ] = 'Oct_Nov_2014'
TS.sig$Start[TS.sig$Interval.start == 139] = 'November2014'
TS.sig$Start[TS.sig$Interval.start > 139 & TS.sig$Interval.start < 294] = 'Nov2014April2015'
TS.sig$Start[TS.sig$Interval.start %in% c(294:341)] = 'April2015'
#TS.sig$Start[TS.sig$Interval.start > 294 & TS.sig$Interval.start < 342] = 'April_June_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(342: 370)] = 'June2015'
#TS.sig$Start[TS.sig$Interval.start > 342 & TS.sig$Interval.start < 371] = 'June_July_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(371: 398)] = 'July2015'
#TS.sig$Start[TS.sig$Interval.start > 371 & TS.sig$Interval.start < 399] = 'July_Aug_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(399: 439)] = 'August2015'
#TS.sig$Start[TS.sig$Interval.start > 399 & TS.sig$Interval.start < 440] = 'Aug_Sept_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(440: 473)] = 'Sept2015'
#TS.sig$Start[TS.sig$Interval.start > 440 & TS.sig$Interval.start < 474] = 'Sept_Oct_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(474: 502)] = 'Oct2015'
#TS.sig$Start[TS.sig$Interval.start > 474 & TS.sig$Interval.start < 503] = 'Oct_Nov_2015'
TS.sig$Start[TS.sig$Interval.start == 503] = 'Nov2015'


TS.sig$End[TS.sig$Interval.end %in% c(0:75)] = 'July2014'
#TS.sig$End[TS.sig$Interval.end > 0 & TS.sig$Interval.end <76] = 'JunetoSept_2014'
TS.sig$End[TS.sig$Interval.end %in% c(76:110)] = 'September2014'
#TS.sig$End[TS.sig$Interval.end > 76 & TS.sig$Interval.end < 111 ] = 'SepttoOct_2014'
TS.sig$End[TS.sig$Interval.end %in% c(111:138)] = 'October2014'
#TS.sig$End[TS.sig$Interval.end > 111 & TS.sig$Interval.end < 139 ] = 'Oct_Nov_2014'
TS.sig$End[TS.sig$Interval.end == 139] = 'November2014'
TS.sig$End[TS.sig$Interval.end > 139 & TS.sig$Interval.end < 294] = 'Nov2014_April2015'
TS.sig$End[TS.sig$Interval.end %in% c(294:341)] = 'April2015'
#TS.sig$End[TS.sig$Interval.end > 294 & TS.sig$Interval.end < 342] = 'April_June_2015'
TS.sig$End[TS.sig$Interval.end %in% c(342: 370)] = 'June2015'
#TS.sig$End[TS.sig$Interval.end > 342 & TS.sig$Interval.end < 371] = 'June_July_2015'
TS.sig$End[TS.sig$Interval.end %in% c(371: 398)] = 'July2015'
#TS.sig$Start[TS.sig$Interval.end > 371 & TS.sig$Interval.end < 399] = 'July_Aug_2015'
TS.sig$End[TS.sig$Interval.end %in% c(399: 439)] = 'August2015'
#TS.sig$End[TS.sig$Interval.end > 399 & TS.sig$Interval.end < 440] = 'Aug_Sept_2015'
TS.sig$End[TS.sig$Interval.end %in% c(440: 473)] = 'Sept2015'
#TS.sig$End[TS.sig$Interval.end > 440 & TS.sig$Interval.end < 474] = 'Sept_Oct_2015'
TS.sig$End[TS.sig$Interval.end %in% c(474: 502)] = 'Oct2015'
#TS.sig$End[TS.sig$Interval.end > 474 & TS.sig$Interval.end < 503] = 'Oct_Nov_2015'
TS.sig$End[TS.sig$Interval.end == 503] = 'Nov2015'





In [ ]:
%%R
Mult_TS = TS.sig %>% group_by(OTU) %>% mutate(TS_num = n()) %>% mutate(diff = Interval.end-Interval.start)

head(Mult_TS) %>% as.data.frame

In [ ]:
%%R
Mult_TS$ISIE = paste(Mult_TS$Start, Mult_TS$End, sep = '-')

In [ ]:
%%R
Mult_TS_sum = Mult_TS %>% group_by(ISIE) %>% summarise(counts = n(), max_diff = max(diff), min_diff = min(diff))

In [ ]:
%%R
Mult_TS$type = ifelse(Mult_TS$diff > 392, 'Long Term',
                      ifelse(Mult_TS$diff %in% c(210: 392), 'Extended',
                            ifelse(Mult_TS$diff %in% c(103: 209), 'Interannual',
                                ifelse(Mult_TS$diff %in% c(30:102), 'Seasonal',
                                       ifelse(Mult_TS$ISIE %in% c('Nov2014April2015-April2015'), 'Seasonal','Short term')))))

In [ ]:
%%R
tax = as.data.frame(tax_table(physeq.Bulk.Sparsity))
tax$OTU = rownames(tax)
head(tax)

In [ ]:
%%R
tax$Rank2 = gsub("__", "", tax$Rank2)
tax$Rank3 = gsub("__", "", tax$Rank3)
tax$Rank4 = gsub("__", "", tax$Rank4)
tax$Rank5 = gsub("__", "", tax$Rank5)
tax$Rank6 = gsub("__", "", tax$Rank6)
tax$Rank7 = gsub('__', "", tax$Rank7)

In [ ]:
%%R
Mult_TS_tax = left_join(Mult_TS, tax, by = "OTU")

In [ ]:
%%R
Mult_TS_tax$Enrich_Status = ifelse(Mult_TS_tax$Area > 0, "Till", "No-Till" )
head(Mult_TS_tax) %>% as.data.frame

In [ ]:
%%R
sing_resp = filter(Mult_TS_tax, TS_num == 1, adjPvalues < 0.01)
print(length((sing_resp$OTU)))
mult_resp = filter(Mult_TS_tax, TS_num > 1)
print(length(unique(mult_resp$OTU)))


In [ ]:
%%R
sing_resp  %>% group_by(type) %>% filter(Area > 0) %>% summarize(nPT = n()) %>% print()
sing_resp  %>% group_by(type) %>% filter(Area < 0) %>% summarize(nNT = n()) %>% print

In [ ]:
%%R
Mult_TS_sum_type = Mult_TS_tax %>% group_by(type, Rank2, ISIE, Enrich_Status) %>%
        summarise(counts = n(), max_diff = max(diff), min_diff = min(diff))

In [ ]:
%%R
cols = c("#417E36",
"#D567D2",
"#58C537",
"#7A79DB",
"#93BB3B",
"#D84891",
"#50D37A",
"#DE4443",
"#3DC1A0",
"#D95821",
"#4DB7D0",
"#E59833",
"#729DDC",
"#C5B432",
"#905D9E",
"#62B264",
"#D64A64",
"#287C76",
"#BB6241",
"#56729D",
"#9C7124",
"#CD93D2",
"#757F26",
"#B95F7D")

In [ ]:
%%R
filter(Mult_TS_sum_type, type == 'Short term')

In [ ]:
%%R -w 1000 -h 500


Mult_TS_sum_type$type = factor(Mult_TS_sum_type$type, levels = Mult_TS_sum_type$type[order(-Mult_TS_sum_type$counts)])

p = ggplot(data = Mult_TS_sum_type, aes(type, counts, fill = Rank2)) + geom_bar(stat='identity') +
        theme(text = element_text(size=18), axis.text.x = element_text(angle=0, vjust=1))  + 
        xlab('Temporal Enrichment Pattern') + ylab('Number of OTUs') + scale_fill_manual(values = cols, name = "Phylum") 

p

In [ ]:
%%R -w 1000 -h 500


Mult_TS_sum_type$type = factor(Mult_TS_sum_type$type, levels = Mult_TS_sum_type$type[order(-Mult_TS_sum_type$counts)])
Mult_TS_sum_type$Enrich_Status = factor(Mult_TS_sum_type$Enrich_Status, levels = Mult_TS_sum_type$Enrich_Status[order(-Mult_TS_sum_type$type)])


p = ggplot(data = Mult_TS_sum_type, aes(type, counts, fill = Enrich_Status)) + geom_bar(stat='identity') +
        theme(text = element_text(size=18), axis.text.x = element_text(angle=0, vjust=1))  + 
        xlab('Temporal Enrichment Pattern') + ylab('Number of OTUs') + scale_fill_manual(values = cols, name = "Enrichment") 

p

In [ ]:
%%R -w 1000 -h 500

Longterm = filter(Mult_TS_sum_type, type == 'Long Term')%>% group_by(ISIE, type, Rank2) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank2)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type) +
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)

p

In [ ]:
%%R -w 1000 -h 500

Mult_TS_sum_type$ISIE = factor(Mult_TS_sum_type$ISIE, levels = Mult_TS_sum_type$ISIE[order(-Mult_TS_sum_type$counts)])

Longterm = filter(Mult_TS_sum_type, type == 'Extended')  %>% group_by(ISIE, type, Rank2) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank2)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)


p

In [ ]:
%%R -w 1000 -h 500


Longterm = filter(Mult_TS_sum_type, type == 'Interannual') %>% group_by(ISIE, type, Rank2) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank2)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)


p

In [ ]:
%%R -w 1000 -h 500


Longterm = filter(Mult_TS_sum_type, type == 'Seasonal') %>% group_by(ISIE, type, Rank2) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank2)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)


p

In [ ]:
%%R -w 400 -h 400

Mult_TS_sum_type$ISIE = factor(Mult_TS_sum_type$ISIE, levels = Mult_TS_sum_type$ISIE[order(-Mult_TS_sum_type$counts)])

Longterm = filter(Mult_TS_sum_type, type == 'Short term')%>% group_by(ISIE, type, Rank2) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank2)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=0, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)


p

In [ ]:
%%R
filter(Mult_TS, ISIE == 'July2014-August2015', type == "Short term") %>% as.data.frame %>% head()

In [ ]:
%%R -w 1000 -h 500
Mult_TS_sum$ISIE = factor(Mult_TS_sum$ISIE, levels = Mult_TS_sum$ISIE[order(-Mult_TS_sum$counts)])
p = ggplot(data = Mult_TS_sum, aes(ISIE, counts)) + geom_bar(stat='identity') +
        theme(text = element_text(size=14), axis.text.x = element_text(angle=90, vjust=1)) 

p

In [ ]:
%%R -w 1000 -h 500
Mult_TS_rest = filter(Mult_TS_sum, ISIE != 'July2014-Nov2015')

p = ggplot(data = Mult_TS_rest, aes(ISIE, counts)) + geom_bar(stat='identity') +
        theme(text = element_text(size=14), axis.text.x = element_text(angle=90, vjust=1)) 

p

In [ ]:
%%R
sessionInfo()